# Data Structure - Using right algorithim and right data structures 


In [1]:
# This cell contains some utility functions to prepare and execute the benchmarks
import timeit
from statistics import median
from random import choice
from string import ascii_uppercase

In [2]:
def random_string(length):
    """Produce a random string made of *length* uppercase ascii characters"""
    return ''.join(choice(ascii_uppercase) for i in range(length))

def print_scaling(stmt, setup, sizes=[10000, 20000, 30000], repeat=False, units='us'):
    """Print scaling information for the statement *stmt*, executed after *setup*.
    
    The *setup* and *stmt* arguments take a template string where "{N}"
    will be replaced as the size of the input.
    
    The *repeat* flags determined if the setup needs to be run between
    each test run.

    For timeit function following are parameters 
    stmt: The statement or code snippet to be timed, provided as a string.
    setup: Optional setup code to be executed once before each repeat, useful for creating necessary objects or initializing variables.
    repeat: The number of times to repeat the timing measurement (default is 5).
    number: The number of times the stmt is executed within each repeat (default is 1,000,000).

    """
    values = []
    for size in sizes:
        if repeat:
            timings = timeit.repeat(stmt.format(N=size),
                                    setup=setup.format(N=size),
                                    number=1, repeat=1000)
            values.append(min(timings))
        else:
            timings = timeit.repeat(stmt.format(N=size),
                                    setup=setup.format(N=size),
                                    number=1000, repeat=3)
            values.append(min(t/1000 for t in timings))
    unit_factor = {'us': 1e6,
                   'ms': 1e3}[units]
    
    print(' | '.join('N = {} t = {:.2f} ({})'.format(n, t * unit_factor, units) for n, t in zip(sizes, values)))

# Lists

In [3]:
print_scaling("collection.pop()", setup="collection = list(range({N}))")

N = 10000 t = 0.03 (us) | N = 20000 t = 0.03 (us) | N = 30000 t = 0.02 (us)


In [4]:
print_scaling('collection.pop(0)', setup='collection = list(range({N}))')

N = 10000 t = 1.05 (us) | N = 20000 t = 3.06 (us) | N = 30000 t = 5.37 (us)


```
Getting element from the beginning will require all successive elements to move. This pop(0) took more time than pop() which will remove only last element

In [5]:
print_scaling('collection.append(1)', setup='collection = list(range({N}))')

N = 10000 t = 0.01 (us) | N = 20000 t = 0.01 (us) | N = 30000 t = 0.01 (us)


In [6]:
print_scaling('collection.insert(0, 1)', setup='collection = list(range({N}))')

N = 10000 t = 5.04 (us) | N = 20000 t = 9.55 (us) | N = 30000 t = 14.18 (us)


``` 
again inserting element to the start of the list will take more time than inserting at the end

In [7]:
setup_code = '''
import random
random.seed(42)

collection = list(range({N}))
'''
print_scaling('collection.index(random.randint(0, {N}))',
              setup=setup_code)

N = 10000 t = 30.61 (us) | N = 20000 t = 61.19 (us) | N = 30000 t = 93.17 (us)


In [8]:
from bisect import bisect_left

def index_bisect(a, x):
    'Locate the leftmost value exactly equal to x'
    i = bisect_left(a, x)
    if i != len(a) and a[i] == x:
        return i
    raise ValueError

setup_code = '''
from __main__ import index_bisect

import random
random.seed(42)

collection = list(range({N}))
'''
    
print_scaling('index_bisect(collection, random.randint(0, {N}))',
              setup=setup_code)

N = 10000 t = 0.48 (us) | N = 20000 t = 0.48 (us) | N = 30000 t = 0.47 (us)


``` 
bisect allows faster searches on sorted arrays. bisect.bisect function can be used on sorted list to find the index to place the element while maintaining the order or sorted array.

# Deque
``` 
Stands for double ended queue. you can access both leftmost and rightmost elements 

In [9]:
print_scaling('collection.pop()',
              setup='from collections import deque; collection = deque(range({N}))')

N = 10000 t = 0.02 (us) | N = 20000 t = 0.02 (us) | N = 30000 t = 0.02 (us)


In [10]:
print_scaling('collection.popleft()',
              setup='from collections import deque; collection = deque(range({N}))')

N = 10000 t = 0.01 (us) | N = 20000 t = 0.01 (us) | N = 30000 t = 0.01 (us)


``` 
popping leftmost element is quite easier and less time consuming than list

In [11]:
print_scaling('collection.append(1)',
              setup='from collections import deque; collection = deque(range({N}))')

N = 10000 t = 0.01 (us) | N = 20000 t = 0.01 (us) | N = 30000 t = 0.01 (us)


In [12]:
print_scaling('collection.appendleft(1)',
              setup='from collections import deque; collection = deque(range({N}))')

N = 10000 t = 0.01 (us) | N = 20000 t = 0.01 (us) | N = 30000 t = 0.01 (us)


``` 
Effectively both append and appendleft is doing the same thing

In [13]:
print_scaling('collection[0]',
              setup='from collections import deque; collection = deque(range({N}))')
print_scaling('collection[{N} - 1]',
              setup='from collections import deque; collection = deque(range({N}))')
print_scaling('collection[int({N}/2)]',
              setup='from collections import deque; collection = deque(range({N}))')

N = 10000 t = 0.01 (us) | N = 20000 t = 0.01 (us) | N = 30000 t = 0.01 (us)
N = 10000 t = 0.01 (us) | N = 20000 t = 0.01 (us) | N = 30000 t = 0.01 (us)
N = 10000 t = 0.09 (us) | N = 20000 t = 0.16 (us) | N = 30000 t = 0.24 (us)


``` 
Still deque comes with its own challenges. accessing elements in between is time consumin

# Dictionaries

Dictionaries are extermely versatile and extensively used in python. Dictionaries are implemented using Hashmaps and are very good for element insertion, deletion and access. All these operations have O(1) time complexity. 

### Hashmap 
Hashmap is a data structure that associates a set of key-value pairs. The principle behind the hashmaps is to assign the specific index to each key so that its associated value can be stored in an array. Index is obtained using hash function. 

Hashmpas are tricky to implement, because they need to handle the collision that happens when two different objects have same hash code. However, all complexities are hidden behind the implementation and default collision resolution works well in most of the real world. 

In [14]:
hash("Hello")

1923960701352315766

In [15]:
def counter_items(items):
    """ Create dictionary of items and thier corresponding counts """
    counter = {}

    for item in items: 
        if item not in counter: 
            counter[item] = 1 
        else: 
            counter[item]+=1 
    
    return counter 

In [16]:
# the same as above can be implemented using collection.defaultdict which can be used to produce dictionary where new key is automatically assigned
# a default value. 
from collections import defaultdict
def counter_defaultdict (items): 
    counter = defaultdict(int)
    for item in items: 
        counter[item]+=1 
    return counter

In [17]:
items = ["r", "o", "h", "i", "t", " ", "a", "b", "h", "i", "s", "h", "e", "k"]

In [ ]:
counter_items(items) 

{'r': 1,
 'o': 1,
 'h': 3,
 'i': 2,
 't': 1,
 ' ': 1,
 'a': 1,
 'b': 1,
 's': 1,
 'e': 1,
 'k': 1}

In [19]:
counter_defaultdict(items)

defaultdict(int,
            {'r': 1,
             'o': 1,
             'h': 3,
             'i': 2,
             't': 1,
             ' ': 1,
             'a': 1,
             'b': 1,
             's': 1,
             'e': 1,
             'k': 1})

In [20]:
# collections also offer another class called Counter which serves the same purpose 
from collections import Counter 
counter = Counter(items)
counter

Counter({'h': 3,
         'i': 2,
         'r': 1,
         'o': 1,
         't': 1,
         ' ': 1,
         'a': 1,
         'b': 1,
         's': 1,
         'e': 1,
         'k': 1})

In [21]:
setup_code = '''
from __main__ import random_string
collection = {{random_string(16) : i for i in range({N})}}
'''

print_scaling('"ITEM" in collection',
              setup=setup_code,
              sizes=[1000, 2000, 3000])

print_scaling('collection["ITEM"] if "ITEM" in collection else None',
              setup=setup_code,
              sizes=[1000, 2000, 3000])

print_scaling('collection["ITEM"] = 0',
              setup=setup_code,
              sizes=[1000, 2000, 3000])

N = 1000 t = 0.01 (us) | N = 2000 t = 0.01 (us) | N = 3000 t = 0.01 (us)
N = 1000 t = 0.01 (us) | N = 2000 t = 0.01 (us) | N = 3000 t = 0.01 (us)
N = 1000 t = 0.01 (us) | N = 2000 t = 0.01 (us) | N = 3000 t = 0.02 (us)


In [22]:
setup_code = '''
from __main__ import random_string
collection = {{random_string(16) : i for i in range({N})}}
'''

print_scaling('"X"  * {N} in collection',
              setup=setup_code,
              sizes=[1000, 2000, 3000])

N = 1000 t = 0.01 (us) | N = 2000 t = 0.01 (us) | N = 3000 t = 0.01 (us)


In [23]:
setup_code = '''
from uuid import uuid4
from collections import Counter
from __main__ import counter_defaultdict, counter_dict
import random

random.seed(42)
collection = [random.randint(0, 100) for i in range({N})]
'''
from collections import defaultdict
def counter_defaultdict(items):
    counter = defaultdict(int)
    for item in items:
        counter[item] += 1
    return counter

def counter_dict(items): 
    counter = {} 
    for item in items: 
        if item not in counter: 
            counter[item] = 0 
        else: 
            counter[item] += 1 
    return counter

print_scaling('Counter(collection)',
              setup=setup_code,
              sizes=[1000, 2000, 3000])

print_scaling('counter_defaultdict(collection)',
              setup=setup_code,
              sizes=[1000, 2000, 3000])

print_scaling('counter_dict(collection)',
              setup=setup_code,
              sizes=[1000, 2000, 3000])

N = 1000 t = 19.53 (us) | N = 2000 t = 34.68 (us) | N = 3000 t = 49.40 (us)
N = 1000 t = 36.73 (us) | N = 2000 t = 64.52 (us) | N = 3000 t = 96.31 (us)
N = 1000 t = 45.19 (us) | N = 2000 t = 85.87 (us) | N = 3000 t = 128.09 (us)


```
We can see Counter is fastest option here

In [24]:
from string import ascii_uppercase
import random
random.seed(42)

strings = [random_string(32) for i in range(10000)]

In [25]:
%timeit next(s for s in strings if s.startswith('AA'))

3.6 μs ± 219 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)


In [30]:
from patricia import trie
string_trie = trie(**{s: 0 for s in strings})
string_trie

trie({'UDAXIHHEXDVXRCSNBACGHQTARGWUWRNH': 0, 'UDADNRGOQBQABHCWJMRZJMEYLBWTGTLM': 0, 'UDTUOUIBAWLTPLYQGIVJLTXIFCAWJFAV': 0, 'UDUSJRVOIMDFHVCTWVABEJCLKDPKQYMG': 0, 'UDMNEXWTBROSESMRMHUZNXXPONHTMMHL': 0, 'UDMDPIMBQRMAPCPLWCQTKJXUBHOSLBUK': 0, 'UDSFQGEJSLDDASWNWWOAVZFJTCCYNSTH': 0, 'UDOXFPACRSDGQRPYEDPVEVRHMANTMNRQ': 0, 'UDCVGMSXHDIVEYPPGVJSDAKYHJFCGFWS': 0, 'UDFOLBKJOQAGQPXJGGTKOAMQBIJAUIMT': 0, 'UDXUAZDEFKEIUCHNWBCVIVLNTFONYIBB': 0, 'UDVINJFZYQIXQJDWHPNPNYTYZQWJSOTB': 0, 'UDZKIJRHHFAFVAJPPWZBPZYCAMPWTBHT': 0, 'UDHMSPVUEOWKVQAXGEZBHSNSBLTCDNTX': 0, 'UITZRYPONXSIKHCIOHYOSTVMKAPKFPGL': 0, 'UIQONHFTWBTFWPTCUZJLNARHGDQDTLYR': 0, 'UIIJTQHKKHSGLJGKQVNIITNVVGYTZEZY': 0, 'UIUJPJPHHIWHLUUTPHNYTPBDVNQYLELH': 0, 'UIVUXHGXKAMSYQYJJHGXQTAFKIPXCVFP': 0, 'UINLRVPQOZJUXONQKCSHXESRNGFOGCDU': 0, 'UIMIWDPHZDVGKRBTTJWWKYWMPUFXMQCC': 0, 'UIFMJBBQQYPVTCOJFAPLFKMXYHMGWQVA': 0, 'UILOIMKKXRRZMOSOQOSLSGISVFBTOQEI': 0, 'UIBEUOCSFPUYFGTVCUAKWZLLRYTIGWRR': 0, 'UIBNVHMNBPDUFVOVRXVVYDGPUAHYNBLI': 0, 'UIGLROHDPSYIOGKTHG

In [31]:
%timeit next(string_trie.iter('AA'))

976 ns ± 5.94 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


In [32]:
setup_code = '''
from itertools import islice
from random import choice, seed, shuffle
from string import ascii_uppercase
seed(42)

# Initialize strings without letter 'A'
strings = [''.join(choice(ascii_uppercase.replace('A', '')) for i in range(32)) for j in range({N})]

# We make sure we only have 10 results that start with 'AA'
strings = ['AA' + s[2:] if i < 10 else s for i, s in enumerate(strings)]

# Randomization
shuffle(strings)
'''

print_scaling('[s for s in strings if s.startswith("AA")]',
              setup=setup_code,
              sizes=[10000, 20000, 30000],
              repeat=False, units='us')

N = 10000 t = 188.73 (us) | N = 20000 t = 385.71 (us) | N = 30000 t = 586.13 (us)


In [33]:
setup_code = '''
from patricia import trie
from random import choice, seed, shuffle
from string import ascii_uppercase
from itertools import islice
seed(42)

# Initialize strings without letter 'A'
strings = [''.join(choice(ascii_uppercase.replace('A', '')) for i in range(32)) for j in range({N})]

# We make sure we only have 10 results that start with 'AA'
strings = ['AA' + s[2:] if i < 10 else s for i, s in enumerate(strings)]

# Randomization
shuffle(strings)
strings_dict = {{s: 0 for s in strings}}
strings_trie = trie(**strings_dict)
'''

print_scaling('list(strings_trie.iter("AA"))',
              setup=setup_code,
              sizes=[10000, 20000, 30000], units='us')

N = 10000 t = 3.44 (us) | N = 20000 t = 3.38 (us) | N = 30000 t = 3.38 (us)


### Building an in-memory search index using hashmap

how to build an invereted index based on dictionary of lists. 

In [26]:
docs = [
    "the cat is under the table", 
    "the dog is under the table", 
    "cats and dogs smell roses", 
    "carla eats an apple"
]

``` 
searching word apple we can use dictionary like 

In [27]:
matches = [doc for doc in docs if "table" in doc]
matches

['the cat is under the table', 'the dog is under the table']

```
the approach is simple and works well for one off queries. However if we need to query the collection quite often. It will be better to optimize the query. Linear scan takes O(N) 

Better approach can be to spend sometime preprocessing the documents so taht they are easier to find at query time. We can build the index called inverted index, that associates each word in our collection with list of document where that word is present. 
For example "table" comes for first 2 sentences, therefore corressponds to 0 and 1 indices. 

In [28]:
# build the index 
index = {} 

for i, doc in enumerate(docs):
    for word in doc.split(): 
        if word not in index: 
            index[word] = [i]
        else: 
            index[word].append(i)

index

{'the': [0, 0, 1, 1],
 'cat': [0],
 'is': [0, 1],
 'under': [0, 1],
 'table': [0, 1],
 'dog': [1],
 'cats': [2],
 'and': [2],
 'dogs': [2],
 'smell': [2],
 'roses': [2],
 'carla': [3],
 'eats': [3],
 'an': [3],
 'apple': [3]}

In [29]:
# getting the document from the index 
results = index["table"]

result_documents = [docs[i] for i in results]
result_documents

['the cat is under the table', 'the dog is under the table']

```
After creation of index, the time complexity to get the document will O(1). With inverted index, we can query any number of documents (as long as they fit in memory) in constant time. Indexing is technique frequently used to retrieve data in search engines and databases.

Inverted index is an expensive operation and requires you to encode every possible query. 

# Sets
```
Are unordered collections of elements, with additional restrictions that the element must be unique. 
The choice where sets are great - during membership tests (testing object is present in the collection) and for union, intersection, difference. 

In python sets are implemented using hash based algorithims. 
Time complexity for addition, deletion and testing a membership is O(1)

In [34]:
x = list (range(1000)) + list(range(500))
x_unique = set(x)

In [35]:
print_scaling('x_unqie = set(x)',
              setup="x = list (range(1000)) + list(range(500))",
              sizes=[10000, 20000, 30000], units='us')

N = 10000 t = 8.91 (us) | N = 20000 t = 8.68 (us) | N = 30000 t = 8.48 (us)


```
removing duplicates from the set is usually O(N) operation as it requires reading all inputs and putting each element in the set. 

Set offers number of operations such as "union", "intersection", and "difference". 

in previous example where we built the inverted indexes, we can use that to find all the documents comprising of cat and table

In [36]:
index = {}

for i, doc in enumerate(docs):
    for word in doc.split():
        if word not in index: 
            index[word] = {i} # define it as set instead of list
        else: 
            index[word].add(i)
index

{'the': {0, 1},
 'cat': {0},
 'is': {0, 1},
 'under': {0, 1},
 'table': {0, 1},
 'dog': {1},
 'cats': {2},
 'and': {2},
 'dogs': {2},
 'smell': {2},
 'roses': {2},
 'carla': {3},
 'eats': {3},
 'an': {3},
 'apple': {3}}

In [37]:
# find the documents that have cat and table 
cat_and_table = index["cat"].intersection(index["table"])
cat_and_table

{0}

In [38]:
[docs[i] for i in cat_and_table]

['the cat is under the table']

# Heaps